# **Collaborative Filtering Recommender System For Movie**
**Mehrdad Mohammadian** - fall 2021

**Dataset:**  MovieLens 100k

# **Import Libs**

In [ ]:
# install surprise lib
! pip install scikit-surprise

In [165]:
import pandas as pd
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from surprise import Reader, Dataset

In [371]:
reader = Reader(rating_scale=(1, 5))

# **Find The Best HyperParameters**

### **Load Dataset**

In [374]:
cols = ['user_id', 'movie_id', 'rating', 'timestamp']
data = pd.read_csv('u.data',  sep='\t', names=cols)
data.drop(columns='timestamp', inplace=True)
data = Dataset.load_from_df(data, reader)

### **Grid Search**

In [381]:
sim_options = {
    "name": ["cosine", "pearson"],
    "user_based": [False, True],
}
bsl_options = {
    'method': ['sgd'],
    'learning_rate': [0.0005, 0.005]
}
param_grid = {
    "sim_options": sim_options,
    'bsl_options': bsl_options
}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing t

### **Results**

In [382]:
gs.best_params

{'mae': {'bsl_options': {'learning_rate': 0.0005, 'method': 'sgd'},
  'sim_options': {'name': 'pearson', 'user_based': False}},
 'rmse': {'bsl_options': {'learning_rate': 0.0005, 'method': 'sgd'},
  'sim_options': {'name': 'cosine', 'user_based': False}}}

# **Train The Model**

### **Load Dataset**

In [384]:
cols = ['user_id', 'movie_id', 'rating', 'timestamp']
data = pd.read_csv('u.data',  sep='\t', names=cols)
data.drop(columns='timestamp', inplace=True)

In [385]:
data.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [504]:
sparse = data.pivot(index='user_id', columns='movie_id', values='rating')

In [505]:
sparse

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [386]:
X = data.copy()
y = data['user_id']
X_train, X_test, _ ,_ = train_test_split(X, y, test_size = 0.20, stratify=y, random_state=42)

In [387]:
print('Data Shape:', X_train.shape, X_test.shape)

Data Shape: (80000, 3) (20000, 3)


In [388]:
X_train = Dataset.load_from_df(X_train[['user_id', 'movie_id', 'rating']], reader)
X_test = Dataset.load_from_df(X_test[['user_id', 'movie_id', 'rating']], reader)

X_train = X_train.build_full_trainset()
X_test = X_test.build_full_trainset()
X_test = X_test.build_testset()

### **Train**

In [391]:
options = {
    'bsl_options': {'learning_rate': 0.0005, 'method': 'sgd'},
    'sim_options': {'name': 'pearson', 'user_based': False}
}
model = KNNWithMeans(sim_options=options['sim_options'], bsl_options=options['bsl_options'])
model.fit(X_train)

Computing the pearson similarity matrix...
Done computing similarity matrix.


### **Test With Metrics**

In [392]:
predictions= model.test(X_test)
accuracy.mae(predictions)
accuracy.rmse(predictions)

MAE:  0.7395
RMSE: 0.9436


0.94358288574682

### **Fill sparse matrix with predicted rates**
after this section we would have a dense matrix

In [506]:
for u_index, row in sparse.iterrows():
    user_num = u_index - 1
    for i_index, item in enumerate(row):
        item_num = i_index+1
        item_rate = sparse.iloc[user_num][item_num]
        if pd.isnull(item_rate):
            sparse.iloc[user_num][item_num] = model.predict(user_num+1, item_num).est
            # print(f'user {user_num+1}  item {item_num} is {item_rate} and predict is {predict.est}')

In [507]:
sparse

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.000000,3.000000,4.000000,3.000000,3.000000,5.000000,4.000000,1.000000,5.000000,3.000000,2.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,4.000000,5.000000,4.000000,1.000000,4.000000,4.000000,3.000000,4.000000,3.000000,2.000000,4.000000,1.000000,3.000000,3.000000,5.000000,4.000000,2.000000,1.000000,2.000000,2.000000,3.000000,4.000000,3.000000,...,3.924803,1.0,4.0,3.527887,3.0,2.0,3.0,3.527887,4.0,2.0,5.0,1.0,3.527887,4.966667,3.0,2.955497,1.0,2.0,1.0,2.5,2.0,3.291346,3.527887,3.527887,3.527887,3.0,2.0,3.0,1.0,2.0,3.0,4.0,3.0,2.0,3.527887,3.527887,3.0,3.527887,3.0,3.0
2,4.000000,3.437731,3.125313,3.591362,3.556621,3.979347,3.967059,4.080176,4.056785,2.000000,3.992702,4.651292,4.000000,4.000000,3.964021,3.373254,2.990827,3.184370,3.000000,3.545306,3.095779,4.568064,4.394812,3.639814,4.000000,3.721893,2.826593,4.112366,2.838313,4.259122,3.685956,3.670273,3.734417,2.425031,2.353681,2.465602,2.172993,3.134759,3.494546,3.039454,...,3.657439,1.0,4.0,3.527887,3.0,2.0,3.0,3.527887,4.0,2.0,5.0,1.0,3.527887,4.543756,3.0,3.188619,1.0,2.0,1.0,2.5,2.0,3.235213,3.527887,3.527887,3.527887,3.0,2.0,3.0,1.0,2.0,3.0,4.0,3.0,2.0,3.527887,3.527887,3.0,3.527887,3.0,3.0
3,3.150601,2.658846,2.492210,2.632303,2.867090,3.066745,3.204607,3.064673,3.248655,2.966963,3.337028,3.527199,2.684429,3.356114,2.972057,2.402100,2.694866,2.016343,2.976986,2.765816,1.985491,3.392172,3.157432,2.943609,2.924515,2.443715,2.492999,3.091800,2.268733,3.299563,3.010507,3.083516,2.825591,1.700231,1.000000,1.081102,2.092080,2.141085,2.880294,2.439039,...,2.759627,1.0,4.0,3.527887,3.0,2.0,3.0,3.527887,4.0,2.0,5.0,1.0,3.527887,2.944405,3.0,2.492056,1.0,2.0,1.0,1.0,2.0,2.143602,3.527887,3.527887,3.527887,3.0,2.0,3.0,1.0,2.0,3.0,4.0,3.0,2.0,3.527887,3.527887,3.0,3.527887,3.0,3.0
4,5.000000,4.143578,3.727402,4.575867,4.419570,5.000000,4.914418,5.000000,4.554399,5.000000,4.000000,4.909718,4.999127,5.000000,4.522477,4.166339,3.847748,4.053322,4.554199,4.787188,4.176276,5.000000,5.000000,4.668323,4.291276,4.697797,4.025663,4.894321,3.611310,4.991224,4.297177,4.429383,4.304245,3.296088,2.290718,3.867720,3.614989,3.641149,3.979670,3.983990,...,3.980268,1.0,4.0,3.527887,3.0,2.0,3.0,3.527887,4.0,2.0,5.0,1.0,3.527887,3.500000,3.0,3.646934,1.0,2.0,1.0,2.5,2.0,3.185185,3.527887,3.527887,3.527887,3.0,2.0,3.0,1.0,2.0,3.0,4.0,3.0,2.0,3.527887,3.527887,3.0,3.527887,3.0,3.0
5,4.000000,3.000000,2.361272,3.116525,2.601415,3.226517,3.419331,3.506003,3.519460,3.276340,3.363945,3.763923,2.871801,3.541435,3.117859,2.852480,4.000000,3.417911,3.797974,3.047711,3.000000,3.406108,3.783626,4.000000,3.000000,3.131722,2.435320,3.397508,4.000000,3.372879,3.003372,3.414511,3.115590,2.223576,1.231215,1.865332,2.135194,2.682744,2.516241,4.000000,...,2.980268,1.0,4.0,3.527887,3.0,2.0,3.0,3.527887,4.0,2.0,5.0,1.0,3.527887,3.500000,3.0,2.646934,1.0,2.0,1.0,2.5,2.0,3.914150,3.527887,3.527887,3.527887,3.0,2.0,3.0,1.0,2.0,3.0,4.0,3.0,2.0,3.527887,3.527887,3.0,3.527887,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.977866,4.037906,4.141668,3.785063,4.483867,4.904866,5.000000,4.680618,5.000000,4.393106,5.000000,5.000000,3.831734,4.410353,5.000000,4.370084,3.906283,4.023313,5.000000,4.186113,3.552699,5.000000,5.000000,4.506293,4.485863,4.523259,4.254684,5.000000,3.631797,5.000000,4.719565,4.171452,4.674088,4.0

# **Recommend movies for a system user**

### **Load movie items dataset**

In [534]:
i_cols = ['movie_id', 'title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item',  sep='|', names=i_cols, encoding='latin-1')
movies.head()

,movie_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [558]:
def recommend(user_number, num_items):
    for i in  sparse.iloc[user_number].sort_values(ascending=False).head(num_items).keys():
        print(movies.iloc[i]['title'])

# recommend top 10 movie item to user 456
recommend(456, 10)

Everyone Says I Love You (1996)
Haunted World of Edward D. Wood Jr., The (1995)
Supercop (1992)
Graduate, The (1967)
Guantanamera (1994)
Nikita (La Femme Nikita) (1990)
Evil Dead II (1987)
Patton (1970)
Indiana Jones and the Last Crusade (1989)
Striptease (1996)


### The End